# Introduction

This notebook demonstrates plotting an image on a map.

1. Find some datasets to load
2. Load pixel data in EPSG:3857 projection, same as used by most web maps
3. Display datasets footptints on a map
4. Display image loaded from these datasets on a map

In [ ]:
from IPython.display import display
import ipyleaflet as L
from ipywidgets import widgets as w
import numpy as np

from datacube import Datacube
from datacube.utils.rio import set_default_rio_config

from odc.ui import show_datasets, mk_image_overlay, with_ui_cbk

## Find Datasets

In this example we are using Sentinel 2A NRT product.

In [ ]:
RGB=('nbar_red', 'nbar_green', 'nbar_blue')
set_default_rio_config(aws={'region_name': 'auto'}, 
                       cloud_defaults=True)

dc = Datacube(env='s2')
qq = dict(lat=(-34, -31), 
          lon=(137, 138.5),
          time='2019-04-10')

dss = dc.find_datasets(product='s2a_nrt_granule', 
                       **qq)

## Load Red/Green/Blue bands

We use 1/20 of the native resolution.

In [ ]:
xx = dc.load(product='s2a_nrt_granule',
        datasets=dss,
        output_crs='EPSG:3857',  # This is what Leaflet uses by default
        measurements=RGB,
        group_by='solar_day',
        resolution=(-200, 200),
        progress_cbk=with_ui_cbk())
xx

### Create Leaflet Map with dataset footprints

In [ ]:
m = show_datasets(dss,
                  style={'opacity': 0.3, 'fillOpacity': 0},
                  width='600px', 
                  height='600px', 
                  scroll_wheel_zoom=True)

### Create Leaflet Image Layer

1. Convert xx to an RGBA image
2. Encode as PNG data
3. Render PNG data to "data uri"
4. Compute image bounds
5. Construct ImageLayer with uri from step 3 and bounds from step 4

JPEG compression can also be used (e.g `fmt="jpeg"`), useful for larger images to reduce notebook size in bytes (use `quality=40` to reduce size further), downside is no opacity support unlike PNG. 

In [ ]:
img_layer = mk_image_overlay(xx, clamp=3000, fmt='png')
m.add_layer(img_layer)

### Add Opacity Control

- Add Vertical Slider to the map
- Dragging slider down lowers opacity of the image layer
- Use of `jslink` from `ipywidgets` ensures that this interactive behaviour will work even on nbviewer

In [ ]:
slider = w.FloatSlider(min=0, max=1, value=1,        # Opacity is valid in [0,1] range
                       orientation='vertical',       # Vertical slider is what we want
                       readout=False,                # no need to show exact value
                       layout=w.Layout(width='2em')) # Fine tune display layout, make it thinner

# Connect slider value to opacity property of the Image Layer
w.jslink((slider, 'value'),         
         (img_layer, 'opacity') )
m.add_control(L.WidgetControl(widget=slider))

### Finally display the map

In [ ]:
display(m)

### Sharing Notebooks Online

Unlike notebooks with `matplotlib` figures, saving notebook with Maps after running it is not enough to have results displayed when sharing notebooks online. You also need to save "Widget State" as a separate step, and you need to make sure that "Widget State" is saved only once within the notebook. There are some unexpected behaviours one have to work around. At the time of writing this only works in `jupyter notebook` not `jupyter lab`.

A robust algorithm for sharing notebooks with widget state is as following:

- Make sure there is no state/output in the notebook
    1. Open notebook in `jupyter notebook` not `lab`
    2. Run Menu: `Kernel > Restart and Clear Output`
    3. Run Menu: `Widgets > Clear Notebook Widget State`
    4. Save notebook
    5. Run Menu: `File > Close and halt`
    6. Verify from filesystem that notebook size is small
- Starting from clean notebook do
    1. Run Menu: `Cell > Run All`
    2. Run Menu: `Widgets > Save Notebook Widget State`
    3. Save notebook
    4. Run Menu: `File > Close and halt`
    
Following procedure above will ensure that widget state is stored in the notebook, and also that only relevant widget state is stored. If you re-run any cells and then save widget state you might end up with stale widget state present in the notebook document as well, that state won't be displayed but will significantly increase your notebook on-disk size, preventing tools like nbviewer from rendering it. The important thing to realise is that `clear widget state` only removes it from the notebook file, not from your browser, so next time you run `save widget state` stale state will re-appear on disk. To make sure that stale `widget state` is properly gone you have to reload the notebook in your browser after clearing it.

------------------------------------------------